# Chapter 6 - Connecting Causality and Deep Learning
### Introductory examples on femur-height model and rock throwing example

The notebook is a code companion to chapter 5 of the book [Causal AI](https://www.manning.com/books/causal-ai) by [Robert Osazuwa Ness](https://www.linkedin.com/in/osazuwa/). <a href="https://colab.research.google.com/github/altdeep/causalML/blob/master/book/chapter%206/chapter_6_femur_height_intro.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

This notebook was written in Google Colab using Python version 3.10.12. It was written in pyro version 1.84, and was recently tested win 1.9.

In [1]:
!pip install pyro-ppl==1.8.4

## Listing 6.1 Pyro pseudocode of the causal graphical model in Figure 6.1

Figure 6.1 in the book is a causal DAG relating femur length to height.

![A DAG mapping femur length to height](https://github.com/altdeep/causalML/blob/master/book/chapter%206/images/Figure%206_1%20femur-height-DAG.png?raw=true)

Here is how we'd model it in Pyro.

In [2]:
from pyro.distributions import Normal
from pyro import sample

def cgm_model():    #A
    x = sample("x", Normal(47., 2.3))    #A
    y = sample("y", Normal(25. + 3*x, 3.3))    #A
    return x, y    #B

#A x and y are sampled from their causal Markov kernels, in this case Normal distributions.
#B Repeatedly calling cgm_model will return samples from P(X, Y).

cgm_model()

(tensor(46.8180), tensor(166.1476))

## Listing 6.2: CGM rewritten as a SCM



We convert this model to an SCM using the following algorithm.
1.  Introduce a new latent cause parent for X called Nx and a new latent causal parent for Y called Ny with distributions P(Nx) and P(Ny)
2.  Make X and Y deterministic functions of Nx and Ny such that P(X, Y) in this new model is the same as the old model.

To convert the CGM to an SCM, we introduce a latent “exogenous” parents, Nx for X and Ny for Y, and probability distributions P(Nx) and P(Ny) for these latents. We then set X and Y deterministically given their parents via functions fx and fy.
We have two new latent variables Nx and Ny with distributions P(Nx) and P(Ny). X and Y each have their own functions fx and fy that that deterministically set X and Y given their parents in the graph. This difference is key; X and Y are generated in the model described in Figure 6.1 but set deterministically in this new model. To emphasize this, I use the assignment operator “:=”, instead of the equal sign “=” to emphasize that fx and fy assign the values of x and y.
To meet our goal of converting our CGM to an SCM, we want P(X) and P(Y|X=x) to be the same across both models. To achieve this, we have to choose P(Nx), P(Ny), fx, and fy such that P(X) is still Normal(47, 2.3) and P(Y|X=x) is still Normal(25 + 3.3x, 3.3). One option is to do a simple reparameterization. Linear functions of normally distributed random variables are also normally distributed. So we can implement the model in Figure 6.3.

![A DAG mapping femur length to height](https://github.com/altdeep/causalML/blob/master/book/chapter%206/images/Figure%206_3_rewrite_as_SCM.png?raw=true)

In code, we rewrite this as:

In [3]:
from pyro.distributions import Normal
from pyro import sample

def scm_model():
    n_x = sample("n_x", Normal(0., 2.3))    #A
    n_y = sample("n_y", Normal(0., 3.3))    #A
    x = 47. + n_x    #B
    y = 25. + 3.*x + n_y    #B
    return x, y    #C

#A We sample these new latent variables from a standard normal distribution.
#B X and Y are calculated deterministically as linear transformations of the n_x and n_y
#C The returned samples of P(X, Y) match the first model.

scm_model()

(tensor(47.6776), tensor(165.9406))

## Listing 6.3: The rock-throwing example from chapter 2 is an SCM

Consider the example our rock-throwing example from chapter 2. In this example, either Jenny or Brian or both throw a rock at window if they are inclined to do so. The window breaks depending on if either or both Jenny and Brian throw and the strength of the windowpane.
How might we convert this model to an SCM? In fact, this model is already an SCM. We captured this with the following code:


In [4]:
import pandas as pd
import random

def true_dgp(jenny_inclination, brian_inclination, window_strength):
    jenny_throws_rock = jenny_inclination > 0.5
    brian_throws_rock = brian_inclination > 0.5
    if jenny_throws_rock and brian_throws_rock:
        strength_of_impact = 0.8
    elif jenny_throws_rock or brian_throws_rock:
        strength_of_impact = 0.6
    else:
        strength_of_impact = 0.0
    window_breaks = window_strength < strength_of_impact
    return jenny_throws_rock, brian_throws_rock, window_breaks

generated_outcome = true_dgp(
    jenny_inclination=random.uniform(0, 1),
    brian_inclination=random.uniform(0, 1),
    window_strength=random.uniform(0, 1)
)
#A The input values are instances of "exogenous" variables.
#B Jenny and Brian throw the rock if so inclined. jenny_throws_rock and brian_throws_rock are endogenous variables.
#B The assignment functions of jenny_throws_rock and brian_throws_rock are both `lambda inclination: inclination > .05`
#C strength_of_impact is an endogenous variable. This entire if-then expression is the assignment function for strength of impact.
#D window_breaks is an endogenous variable. The assignment function is `lambda strength_of_impact, window_strength: strength_of_impact > window_strength`
#E Each exogenous variable has a Uniform(0, 1) distribution.

generated_outcome

(False, False, False)